In [9]:
from collections import defaultdict

def input_grammar():
    grammar = defaultdict(list)
    num_rules = int(input("Enter the number of production rules: "))

    print("Enter the grammar rules:")
    for _ in range(num_rules):
        rule = input().strip()
        lhs, rhs = rule.split("->")
        lhs = lhs.strip()
        rhs_productions = rhs.split("|")
        grammar[lhs] = [prod.strip() for prod in rhs_productions]

    return grammar

def compute_first(grammar):
    first = defaultdict(set)

    def find_first(symbol):
        if not symbol.isupper():
            return {symbol}
        if first[symbol]:
            return first[symbol]

        for production in grammar[symbol]:
            if production == "":  # Epsilon
                first[symbol].add("")
            else:
                for char in production:
                    first[symbol].update(find_first(char))
                    if "" not in find_first(char):  # Stop if epsilon is not in First(char)
                        break

        return first[symbol]

    for non_terminal in grammar:
        find_first(non_terminal)

    return first

def compute_follow(grammar, start_symbol, first):
    follow = defaultdict(set)
    follow[start_symbol].add('$')

    while True:
        updated = False

        for lhs in grammar:
            for production in grammar[lhs]:
                for i in range(len(production)):
                    if production[i].isupper():  # it's non-terminal
                        #There is a next symbol
                        if i + 1 < len(production):
                            next_symbol = production[i + 1]

                            if next_symbol.isupper():
                                # Add First(next_symbol) to Follow
                                before_update = len(follow[production[i]])
                                follow[production[i]].update(first[next_symbol] - {""})

                                if len(follow[production[i]]) > before_update:
                                    updated = True
                            else:
                                # Next symbol terminal add to Follow
                                before_update = len(follow[production[i]])
                                follow[production[i]].add(next_symbol)

                                if len(follow[production[i]]) > before_update:
                                    updated = True

                        #At the end has epsilon
                        if i + 1 == len(production) or (next_symbol.isupper() and "" in first[next_symbol]):
                            before_update = len(follow[production[i]])
                            follow[production[i]].update(follow[lhs])

                            if len(follow[production[i]]) > before_update:
                                updated = True

        if not updated:
            break

    return follow


In [10]:
def main():
    grammar = input_grammar()
    start_symbol = input("Enter the start symbol: ").strip()

    first = compute_first(grammar)
    follow = compute_follow(grammar, start_symbol, first)

    print("\nFirst Sets:")
    for non_terminal in first:
        print(f"First({non_terminal}) = {first[non_terminal]}")

    print("\nFollow Sets:")
    for non_terminal in follow:
        print(f"Follow({non_terminal}) = {follow[non_terminal]}")


if __name__ == "__main__":
    main()

Enter the number of production rules: 3
Enter the grammar rules:
S->Bb|Cd  
B->aB|  
C->cC|
Enter the start symbol: S

First Sets:
First(S) = {'', 'c', 'b', 'd', 'a'}
First(B) = {'', 'a'}
First(C) = {'', 'c'}

Follow Sets:
Follow(S) = {'$'}
Follow(B) = {'b'}
Follow(C) = {'d'}
